In [1]:
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

drive.mount('/gdrive') 
data = pd.read_csv('/gdrive/My Drive/DecisionTree/Capstone/Clean_Client_Spend_Data_v2.csv')

Mounted at /gdrive


<ipython-input-1-62df1ba0e0d1>:12: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/gdrive/My Drive/DecisionTree/Capstone/Clean_Client_Spend_Data_v2.csv')


In [2]:
#Define payment duration
from datetime import datetime
data['Payment_Duration'] = pd.to_datetime(data['PAYMENT_DATE']) -pd.to_datetime(data['INVOICE_DATE'])

In [3]:
mean_duration = data['Payment_Duration'].mean()
data['Payment_Duration'] = data['Payment_Duration'].fillna(mean_duration).astype('timedelta64[D]').astype('int')
data['Payment_Duration']

0           3
1          27
2         153
3          29
4           3
         ... 
715892     84
715893      0
715894    116
715895    117
715896     60
Name: Payment_Duration, Length: 715897, dtype: int64

In [4]:
data

,UNIQUE_INVOICE_ID,INVOICE_DATE,INVOICE_RECORD_CREATE_DATE,PAYMENT_DATE,INVOICE_QUANTITY,INVOICE_GL_NAME,SPEND,SPEND_CURRENCY,PO_TYPE,PO_NUMBER,...,PO_NON_PO_FLAG,SUPPLIER_ID,SUPPLIER_PARENT_NAME,SUPPLIER_NORMALIZED_NAME,SUPPLIER_NAME,IBM_CATEGORY,IBM_SUB_CATEGORY,IBM_COMMODITY,CLIENT_ID,Payment_Duration
0,ISV44180:7704-00144:SE.2110.220823.09.004316:2...,2022-08-22,2022-10-25,2022-08-25,NaN,Construction in progress,-3.367216e+07,EUR,NaN,NaN,...,NONPO,ISV44180,INTEROGO FOUNDATION,LOGEVO,LOGEVO AB,Corporate and Prof Svcs,Uncategorized,Uncategorized,CLIENT3,3
1,ISV00018:13500210200912:SE.2120.211210.09.0059...,2021-11-26,2022-01-11,2021-12-23,NaN,Clearing account others,-2.451456e+07,EUR,NaN,NaN,...,NONPO,ISV00018,CAPGEMINI,CAPGEMINI,CAPGEMINI SVERIGE AB SEK,Information Technology,IT Services,IT Maintenance Services,CLIENT3,27
2,TCCC:2021:0801:0001028343:5100021408:002,2021-06-01,2022-02-04,2021-11-01,0.0,SAP Goods Receipt/Invoice Receipt Clearing - PO,-2.142774e+07,USD,K,7001600293,...,PO,1028343,MICROSOFT,MICROSOFT,MICROSOFT CORP,Information Technology,IT Software,Software Licenses,CLIENT2,153
3,ISV19233:1022446155:DE.1206.220215.09.004679:2...,2022-02-14,2022-07-20,2022-03-15,NaN,Linear TV / Film,-1.174494e+07,EUR,NaN,NaN,...,NONPO,ISV19233,WPP,MEDIACOM,MEDIACOM AGENTUR FUER MEDIABERATUNG GMBH,Marketing & Communications,Advertising & Creative,Advertising & Creative Agency Fee,CLIENT3,29
4,TCCC:2022:0101:1800014834:1900429853:001,2022-09-19,2022-10-19,2022-09-22,0.0,Interface Clearing,-1.165119e+07,USD,NaN,NaN,...,NONPO,1800014834,CUTRALE AUBURNDALE STO,CUTRALE AUBURNDALE STO,CUTRALE AUBURNDALE STO,Corporate and Prof Svcs,Business & Financial,Other Financial Services,CLIENT2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715892,HQ179-0000071875:1:71009:101,2021-11-10,2022-03-08,2022-02-02,NaN,Prepaid Miscellaneous Expenses,3.029980e+07,USD,NaN,213013,...,PO,71009,COGNIZANT TECHNOLOGY SOLUTIONS,COGNIZANT,"COGNIZANT TRIZETTO SOFTWARE GROUP, INC",Information Technology,IT Software,Software Maintenance,CLIENT4,84
715893,CATALYST:1700630716:000001:::2022::P0001000:00...,2022-07-06,2022-10-11,2022-07-06,NaN,ACCTS PAYABLE TRADE,4.628361e+07,USD,NaN,NaN,...,NONPO,6013816,NEWEDGE,NEWEDGE,NEWEDGE USA LLC,Corporate and Prof Svcs,Business & Financial,Other Financial Services,CLIENT1,0
715894,TCCC:2021:0801:0001547707:5100033201:002,2021-12-09,2022-05-19,2022-04-04,25600000.0,SAP Goods Receipt/Invoice Receipt Clearing - PO,4.640000e+07,USD,K,7001638864,...,PO,1547707,BAIN,BAIN AND COMPANY,BAIN & COMPANY INC,Corporate and Prof Svcs,CPS Consulting,Management Consulting,CLIENT2,116
715895,TCCC:2022:1678:0001198906:5100003684:005,2022-03-02,2022-07-25,2022-06-27,23940.0,SAP Goods Receipt/Invoice Receipt Clearing - PO,6.664896e+07,USD,NaN,4509104300,...,PO,1198906,TAMPA JUICE SERVICE,TAMPA JUICE SERVICE,TAMPA JUICE SERVICE INC,Facilities Management,Facilities Services,Food Services,CLIENT2,117


In [5]:
# Filter suppliers that appear at least 30 times to remove outliers 
filtered_data = data.groupby(["CLIENT_ID", "SUPPLIER_NORMALIZED_NAME"]).filter(lambda x: len(x) >= 30)

In [6]:
# Rank the top3 Spending partitioned by client and commodity and show the info 
filtered_data['rank'] = filtered_data.groupby(['CLIENT_ID'])['SPEND'].rank(ascending=False)
filtered_data_rank = filtered_data[filtered_data['rank'] <= 3]
filtered_data_rank


,UNIQUE_INVOICE_ID,INVOICE_DATE,INVOICE_RECORD_CREATE_DATE,PAYMENT_DATE,INVOICE_QUANTITY,INVOICE_GL_NAME,SPEND,SPEND_CURRENCY,PO_TYPE,PO_NUMBER,...,SUPPLIER_ID,SUPPLIER_PARENT_NAME,SUPPLIER_NORMALIZED_NAME,SUPPLIER_NAME,IBM_CATEGORY,IBM_SUB_CATEGORY,IBM_COMMODITY,CLIENT_ID,Payment_Duration,rank
715265,CATALYST:1700371046:000001:::2022::P0001000:00...,2022-04-19,2022-07-25,2022-04-19,NaN,ACCTS PAYABLE TRADE,1.566284e+06,USD,NaN,NaN,...,6092972,ENGIE,ENGIE,ENGIE INSIGHT SERVICES DBA ENGIE IM,Facilities Maint. & Ops,Utilities,Energy Supply,CLIENT1,0,3.0
715518,CATALYST:1700446983:000001:5115655527:00000000...,2022-05-10,2022-07-25,NaN,NaN,ACCTS PAYABLE TRADE,2.249956e+06,USD,ZARB,4000037154,...,6046929,PECO INSPX,PECO,PECO PALLET INC,Operating Supplies,Product Storage & Delivery,Pallets,CLIENT1,60,2.0
715773,CATALYST:1700734442:000001:::2022::P0001000:00...,2022-08-08,2022-10-11,2022-08-09,NaN,ACCTS PAYABLE TRADE,4.295447e+06,USD,NaN,NaN,...,6092972,ENGIE,ENGIE,ENGIE INSIGHT SERVICES DBA ENGIE IM,Facilities Maint. & Ops,Utilities,Energy Supply,CLIENT1,1,1.0
715802,TCCC:2021:0101:0001518408:5100391215:002,2021-11-02,2022-02-04,2021-12-28,23568733.0,SAP Goods Receipt/Invoice Receipt Clearing - PO,5.134606e+06,USD,F,7001606881,...,1518408,THE INTERPUBLIC GROUP OF COMPANIES,MCCANN,UNIVERSAL MCCANN WORLDWIDE INC,Marketing & Communications,Advertising & Creative,Advertising & Creative Agency Fee,CLIENT2,56,3.0
715841,ISV53342:400007161:BE.1600.220316.09.006287:20...,2022-03-08,2022-07-20,2022-04-06,NaN,External service provider (ESP) cost,6.850996e+06,EUR,NaN,NaN,...,ISV53342,XPO LOGISTICS,XPO,XPO SUPPLY CHAIN NETHERLANDS BV,Logistics,Small Package,Parcel Services,CLIENT3,29,3.0
715853,TCCC:2022:1015:0001627602:5100001220:002,2022-04-12,2022-05-19,2022-04-19,589598101.0,Paid Media - Online - RU GAAP,9.093235e+06,USD,F,7001693614,...,1627602,WPP,MEDIACOM,MEDIACOM LLC,Marketing & Communications,Media,Media Agency Fee,CLIENT2,7,2.0
715863,ISV45802:102300151:NL.1110.221007.09.007770:20...,2022-10-06,2022-12-07,2022-11-03,NaN,Other external expenses,1.100597e+07,EUR,NaN,NaN,...,ISV45802,INTEROGO FOUNDATION,IKEA,INTER IKEA SYSTEMS BV,MRO,MRO Supplies,Electrical,CLIENT3,28,2.0
715867,TCCC:2022:0101:0001192842:5100365328:002,2022-09-08,2022-10-19,2022-09-28,1851827.0,SAP Goods Receipt/Invoice Receipt Clearing - PO,1.257337e+07,USD,NaN,4509395353,...,1192842,CUTRALE NORTH AMERICA,CITRUS PRODUCTS,CITRUS PRODUCTS INC,Facilities Management,Facilities Services,Food Services,CLIENT2,20,1.0
715873,ISV19233:1022460074:DE.1206.221007.09.006010:2...,2022-10-06,2022-12-07,2022-11-04,NaN,Prepaid expenses,1.471320e+07,EUR,NaN,NaN,...,ISV19233,WPP,MEDIACOM,MEDIACOM AGENTUR FUER MEDIABERATUNG GMBH,Marketing & Communications,Media,Media Agency Fee,CLIENT3,29,1.0
715889,1035268134-9:1:54958:101,2022-10-14,2022-12-05,2022-11-28,NaN,Other Professional Fees,2.386722e+07,USD,NaN,201450,...,54958,PRICEWATERHOUSECOOPERS,PRICE WATERHOUSE COOPERS,PRICEWATERHOUSECOOPERS LLP,Corporate and Prof Svcs,CPS Consulting,Management Consulting,CLIENT4,45,3.0


In [13]:
# Extract the potential max payment duration acceptance
commodity = list(filtered_data_rank['IBM_COMMODITY'].unique())
temp_data = filtered_data[filtered_data['IBM_COMMODITY'].isin(commodity)].groupby(['IBM_COMMODITY', 'SUPPLIER_NORMALIZED_NAME'])['Payment_Duration'].mean().reset_index()
temp_data_sorted = temp_data.sort_values(by=['IBM_COMMODITY', 'Payment_Duration'], ascending=[True, False])
result = temp_data_sorted.groupby('IBM_COMMODITY').head(1)
result = result.rename(columns={'Payment_Duration': 'Max_payment_duration', 'SUPPLIER_NORMALIZED_NAME': 'Recommended_Supplier'})

In [14]:
result['Max_payment_duration'] = result['Max_payment_duration'].round().astype(int)
result

,IBM_COMMODITY,Recommended_Supplier,Max_payment_duration
10,Advertising & Creative Agency Fee,DHINGRA ADVERTISERS,393
93,Electrical,EATON,117
182,Energy Supply,ENDESA ENERGIA,126
284,Food Services,FIRST FFC MORANGIS,310
530,Management Consulting,ORANGE,146
586,Media Agency Fee,INTEGRAL AD SCIENCE,123
613,Pallets,NOR CAL BEVERAGE COMPANY,134
643,Parcel Services,IWT GLOBAL SOLUTIONS,120
723,Software Maintenance,VERIZON,121


In [15]:
#combine the data
final_data = pd.merge(filtered_data_rank, result[['IBM_COMMODITY', 'Recommended_Supplier', 'Max_payment_duration']], on='IBM_COMMODITY', how='left')
final_data

,UNIQUE_INVOICE_ID,INVOICE_DATE,INVOICE_RECORD_CREATE_DATE,PAYMENT_DATE,INVOICE_QUANTITY,INVOICE_GL_NAME,SPEND,SPEND_CURRENCY,PO_TYPE,PO_NUMBER,...,SUPPLIER_NORMALIZED_NAME,SUPPLIER_NAME,IBM_CATEGORY,IBM_SUB_CATEGORY,IBM_COMMODITY,CLIENT_ID,Payment_Duration,rank,Recommended_Supplier,Max_payment_duration
0,CATALYST:1700371046:000001:::2022::P0001000:00...,2022-04-19,2022-07-25,2022-04-19,NaN,ACCTS PAYABLE TRADE,1.566284e+06,USD,NaN,NaN,...,ENGIE,ENGIE INSIGHT SERVICES DBA ENGIE IM,Facilities Maint. & Ops,Utilities,Energy Supply,CLIENT1,0,3.0,ENDESA ENERGIA,126
1,CATALYST:1700446983:000001:5115655527:00000000...,2022-05-10,2022-07-25,NaN,NaN,ACCTS PAYABLE TRADE,2.249956e+06,USD,ZARB,4000037154,...,PECO,PECO PALLET INC,Operating Supplies,Product Storage & Delivery,Pallets,CLIENT1,60,2.0,NOR CAL BEVERAGE COMPANY,134
2,CATALYST:1700734442:000001:::2022::P0001000:00...,2022-08-08,2022-10-11,2022-08-09,NaN,ACCTS PAYABLE TRADE,4.295447e+06,USD,NaN,NaN,...,ENGIE,ENGIE INSIGHT SERVICES DBA ENGIE IM,Facilities Maint. & Ops,Utilities,Energy Supply,CLIENT1,1,1.0,ENDESA ENERGIA,126
3,TCCC:2021:0101:0001518408:5100391215:002,2021-11-02,2022-02-04,2021-12-28,23568733.0,SAP Goods Receipt/Invoice Receipt Clearing - PO,5.134606e+06,USD,F,7001606881,...,MCCANN,UNIVERSAL MCCANN WORLDWIDE INC,Marketing & Communications,Advertising & Creative,Advertising & Creative Agency Fee,CLIENT2,56,3.0,DHINGRA ADVERTISERS,393
4,ISV53342:400007161:BE.1600.220316.09.006287:20...,2022-03-08,2022-07-20,2022-04-06,NaN,External service provider (ESP) cost,6.850996e+06,EUR,NaN,NaN,...,XPO,XPO SUPPLY CHAIN NETHERLANDS BV,Logistics,Small Package,Parcel Services,CLIENT3,29,3.0,IWT GLOBAL SOLUTIONS,120
5,TCCC:2022:1015:0001627602:5100001220:002,2022-04-12,2022-05-19,2022-04-19,589598101.0,Paid Media - Online - RU GAAP,9.093235e+06,USD,F,7001693614,...,MEDIACOM,MEDIACOM LLC,Marketing & Communications,Media,Media Agency Fee,CLIENT2,7,2.0,INTEGRAL AD SCIENCE,123
6,ISV45802:102300151:NL.1110.221007.09.007770:20...,2022-10-06,2022-12-07,2022-11-03,NaN,Other external expenses,1.100597e+07,EUR,NaN,NaN,...,IKEA,INTER IKEA SYSTEMS BV,MRO,MRO Supplies,Electrical,CLIENT3,28,2.0,EATON,117
7,TCCC:2022:0101:0001192842:5100365328:002,2022-09-08,2022-10-19,2022-09-28,1851827.0,SAP Goods Receipt/Invoice Receipt Clearing - PO,1.257337e+07,USD,NaN,4509395353,...,CITRUS PRODUCTS,CITRUS PRODUCTS INC,Facilities Management,Facilities Services,Food Services,CLIENT2,20,1.0,FIRST FFC MORANGIS,310
8,ISV19233:1022460074:DE.1206.221007.09.006010:2...,2022-10-06,2022-12-07,2022-11-04,NaN,Prepaid expenses,1.471320e+07,EUR,NaN,NaN,...,MEDIACOM,MEDIACOM AGENTUR FUER MEDIABERATUNG GMBH,Marketing & Communications,Media,Media Agency Fee,CLIENT3,29,1.0,INTEGRAL AD SCIENCE,123
9,1035268134-9:1:54958:101,2022-10-14,2022-12-05,2022-11-28,NaN,Other Professional Fees,2.386722e+07,USD,NaN,201450,...,PRICE WATERHOUSE COOPERS,PRICEWATERHOUSECOOPERS LLP,Corporate and Prof Svcs,CPS Consulting,Management Consulting,CLIENT4,45,3.0,ORANGE,146


In [16]:
#Calculate money saved
final_data['Money_saved'] =final_data['SPEND']*0.05*final_data['Max_payment_duration'] - final_data['Payment_Duration']/365
final_data

,UNIQUE_INVOICE_ID,INVOICE_DATE,INVOICE_RECORD_CREATE_DATE,PAYMENT_DATE,INVOICE_QUANTITY,INVOICE_GL_NAME,SPEND,SPEND_CURRENCY,PO_TYPE,PO_NUMBER,...,SUPPLIER_NAME,IBM_CATEGORY,IBM_SUB_CATEGORY,IBM_COMMODITY,CLIENT_ID,Payment_Duration,rank,Recommended_Supplier,Max_payment_duration,Money_saved
0,CATALYST:1700371046:000001:::2022::P0001000:00...,2022-04-19,2022-07-25,2022-04-19,NaN,ACCTS PAYABLE TRADE,1.566284e+06,USD,NaN,NaN,...,ENGIE INSIGHT SERVICES DBA ENGIE IM,Facilities Maint. & Ops,Utilities,Energy Supply,CLIENT1,0,3.0,ENDESA ENERGIA,126,9.867586e+06
1,CATALYST:1700446983:000001:5115655527:00000000...,2022-05-10,2022-07-25,NaN,NaN,ACCTS PAYABLE TRADE,2.249956e+06,USD,ZARB,4000037154,...,PECO PALLET INC,Operating Supplies,Product Storage & Delivery,Pallets,CLIENT1,60,2.0,NOR CAL BEVERAGE COMPANY,134,1.507471e+07
2,CATALYST:1700734442:000001:::2022::P0001000:00...,2022-08-08,2022-10-11,2022-08-09,NaN,ACCTS PAYABLE TRADE,4.295447e+06,USD,NaN,NaN,...,ENGIE INSIGHT SERVICES DBA ENGIE IM,Facilities Maint. & Ops,Utilities,Energy Supply,CLIENT1,1,1.0,ENDESA ENERGIA,126,2.706131e+07
3,TCCC:2021:0101:0001518408:5100391215:002,2021-11-02,2022-02-04,2021-12-28,23568733.0,SAP Goods Receipt/Invoice Receipt Clearing - PO,5.134606e+06,USD,F,7001606881,...,UNIVERSAL MCCANN WORLDWIDE INC,Marketing & Communications,Advertising & Creative,Advertising & Creative Agency Fee,CLIENT2,56,3.0,DHINGRA ADVERTISERS,393,1.008950e+08
4,ISV53342:400007161:BE.1600.220316.09.006287:20...,2022-03-08,2022-07-20,2022-04-06,NaN,External service provider (ESP) cost,6.850996e+06,EUR,NaN,NaN,...,XPO SUPPLY CHAIN NETHERLANDS BV,Logistics,Small Package,Parcel Services,CLIENT3,29,3.0,IWT GLOBAL SOLUTIONS,120,4.110598e+07
5,TCCC:2022:1015:0001627602:5100001220:002,2022-04-12,2022-05-19,2022-04-19,589598101.0,Paid Media - Online - RU GAAP,9.093235e+06,USD,F,7001693614,...,MEDIACOM LLC,Marketing & Communications,Media,Media Agency Fee,CLIENT2,7,2.0,INTEGRAL AD SCIENCE,123,5.592340e+07
6,ISV45802:102300151:NL.1110.221007.09.007770:20...,2022-10-06,2022-12-07,2022-11-03,NaN,Other external expenses,1.100597e+07,EUR,NaN,NaN,...,INTER IKEA SYSTEMS BV,MRO,MRO Supplies,Electrical,CLIENT3,28,2.0,EATON,117,6.438495e+07
7,TCCC:2022:0101:0001192842:5100365328:002,2022-09-08,2022-10-19,2022-09-28,1851827.0,SAP Goods Receipt/Invoice Receipt Clearing - PO,1.257337e+07,USD,NaN,4509395353,...,CITRUS PRODUCTS INC,Facilities Management,Facilities Services,Food Services,CLIENT2,20,1.0,FIRST FFC MORANGIS,310,1.948873e+08
8,ISV19233:1022460074:DE.1206.221007.09.006010:2...,2022-10-06,2022-12-07,2022-11-04,NaN,Prepaid expenses,1.471320e+07,EUR,NaN,NaN,...,MEDIACOM AGENTUR FUER MEDIABERATUNG GMBH,Marketing & Communications,Media,Media Agency Fee,CLIENT3,29,1.0,INTEGRAL AD SCIENCE,123,9.048618e+07
9,1035268134-9:1:54958:101,2022-10-14,2022-12-05,2022-11-28,NaN,Other Professional Fees,2.386722e+07,USD,NaN,201450,...,PRICEWATERHOUSECOOPERS LLP,Corporate and Prof Svcs,CPS Consulting,Management Consulting,CLIENT4,45,3.0,ORANGE,146,1.742307e+08


In [17]:
client_commodities = {}
for client, commodity, supplier, money in final_data[['CLIENT_ID', 'IBM_COMMODITY', 'Recommended_Supplier', 'Money_saved']].sort_values(by='CLIENT_ID').itertuples(index=False):
    if client not in client_commodities:
        client_commodities[client] = set()
    if commodity not in client_commodities[client]:
        print(f"Client {client} should purchase {commodity} from {supplier}, and will save {money:.2f} dollars per day.")
        client_commodities[client].add(commodity)

Client CLIENT1 should purchase Energy Supply from ENDESA ENERGIA, and will save 9867586.42 dollars per day.
Client CLIENT1 should purchase Pallets from NOR CAL BEVERAGE COMPANY, and will save 15074707.30 dollars per day.
Client CLIENT2 should purchase Advertising & Creative Agency Fee from DHINGRA ADVERTISERS, and will save 100895012.25 dollars per day.
Client CLIENT2 should purchase Media Agency Fee from INTEGRAL AD SCIENCE, and will save 55923395.45 dollars per day.
Client CLIENT2 should purchase Food Services from FIRST FFC MORANGIS, and will save 194887266.10 dollars per day.
Client CLIENT3 should purchase Parcel Services from IWT GLOBAL SOLUTIONS, and will save 41105978.10 dollars per day.
Client CLIENT3 should purchase Electrical from EATON, and will save 64384951.63 dollars per day.
Client CLIENT3 should purchase Media Agency Fee from INTEGRAL AD SCIENCE, and will save 90486179.92 dollars per day.
Client CLIENT4 should purchase Management Consulting from ORANGE, and will save 17

In [18]:
client_id = input("Enter the client ID: ")

# Then, filter the data based on the client ID
client_data = final_data.loc[final_data['CLIENT_ID'] == client_id]

# Check if there is any data for the given client ID
if client_data.empty:
    print(f"No data found for client {client_id}.")
else:
    # Iterate through the unique commodities for the client
    for commodity in client_data['IBM_COMMODITY'].unique():
        # Filter the data for the current commodity and client ID
        commodity_data = client_data.loc[client_data['IBM_COMMODITY'] == commodity]
        # Get the recommended supplier and money saved
        supplier, money = commodity_data[['Recommended_Supplier', 'Money_saved']].iloc[0]
        # Print the recommendation
        print(f"Client {client_id} should purchase {commodity} from {supplier}, and will save {money:.2f} dollars per day.")

Enter the client ID: CLIENT1
Client CLIENT1 should purchase Energy Supply from ENDESA ENERGIA, and will save 9867586.42 dollars per day.
Client CLIENT1 should purchase Pallets from NOR CAL BEVERAGE COMPANY, and will save 15074707.30 dollars per day.
